In [0]:
data = [
    (101, "2023-01-01", "Electronics", "Mobile", 2, 15000),
    (102, "2023-01-02", "Furniture", "Chair", 1, 3500),
    (103, "2023-01-03", "Electronics", "Laptop", 1, 55000),
    (104, "2023-01-04", "Clothing", "Shirt", 3, 1200),
    (105, "2023-01-05", "Clothing", "Jeans", 2, 2000)
]

columns = ["order_id", "order_date", "category", "product", "quantity", "price"]

df = spark.createDataFrame(data, columns)
df.show()


+--------+----------+-----------+-------+--------+-----+
|order_id|order_date|   category|product|quantity|price|
+--------+----------+-----------+-------+--------+-----+
|     101|2023-01-01|Electronics| Mobile|       2|15000|
|     102|2023-01-02|  Furniture|  Chair|       1| 3500|
|     103|2023-01-03|Electronics| Laptop|       1|55000|
|     104|2023-01-04|   Clothing|  Shirt|       3| 1200|
|     105|2023-01-05|   Clothing|  Jeans|       2| 2000|
+--------+----------+-----------+-------+--------+-----+



In [0]:
spark


In [0]:
df_clean = df.dropna() \
    .filter(df.quantity > 0) \
    .filter(df.price > 0)

df_clean.show()


+--------+----------+-----------+-------+--------+-----+
|order_id|order_date|   category|product|quantity|price|
+--------+----------+-----------+-------+--------+-----+
|     101|2023-01-01|Electronics| Mobile|       2|15000|
|     102|2023-01-02|  Furniture|  Chair|       1| 3500|
|     103|2023-01-03|Electronics| Laptop|       1|55000|
|     104|2023-01-04|   Clothing|  Shirt|       3| 1200|
|     105|2023-01-05|   Clothing|  Jeans|       2| 2000|
+--------+----------+-----------+-------+--------+-----+



In [0]:
from pyspark.sql.functions import col

df_transformed = df_clean.withColumn(
    "total_amount",
    col("quantity") * col("price")
)

df_transformed.show()


+--------+----------+-----------+-------+--------+-----+------------+
|order_id|order_date|   category|product|quantity|price|total_amount|
+--------+----------+-----------+-------+--------+-----+------------+
|     101|2023-01-01|Electronics| Mobile|       2|15000|       30000|
|     102|2023-01-02|  Furniture|  Chair|       1| 3500|        3500|
|     103|2023-01-03|Electronics| Laptop|       1|55000|       55000|
|     104|2023-01-04|   Clothing|  Shirt|       3| 1200|        3600|
|     105|2023-01-05|   Clothing|  Jeans|       2| 2000|        4000|
+--------+----------+-----------+-------+--------+-----+------------+



In [0]:
from pyspark.sql.functions import sum

sales_summary = df_transformed.groupBy("category") \
    .agg(sum("total_amount").alias("total_sales"))

sales_summary.show()


+-----------+-----------+
|   category|total_sales|
+-----------+-----------+
|Electronics|      85000|
|  Furniture|       3500|
|   Clothing|       7600|
+-----------+-----------+



In [0]:
sales_summary.createOrReplaceTempView("sales")

spark.sql("""
SELECT category, total_sales
FROM sales
ORDER BY total_sales DESC
""").show()


+-----------+-----------+
|   category|total_sales|
+-----------+-----------+
|Electronics|      85000|
|   Clothing|       7600|
|  Furniture|       3500|
+-----------+-----------+

